In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import gc

%matplotlib inline

In [7]:
plt.rc('figure',dpi=100);

In [8]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos['denominacion_empresa']=avisos['denominacion_empresa'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [37]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante
0,1111780242,YjVJQ6Z
1,1112263876,BmVpYoR
2,1112327963,wVkBzZd
3,1112318643,OqmP9pv
4,1111903673,DrpbXDP


In [10]:
vistas_por_area=vistas.copy()
vistas_por_area['vistas_por_area']=0;
vistas_por_area=pd.merge(vistas_por_area,avisos,on='idaviso', how='left')
vistas_por_area=vistas_por_area.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_por_area.head()

,idpostulante,nombre_area,vistas_por_area
0,0005E,Tecnologia / Sistemas,2
1,00Lkv,Comercio Exterior,2
2,00dMd,Comercio Exterior,4
3,01QAq,Contabilidad,1
4,021OM,Comercio Exterior,5


In [11]:
vistas_por_nivel=vistas.copy()
vistas_por_nivel['vistas_por_nivel']=0;
vistas_por_nivel=pd.merge(vistas_por_nivel,avisos,on='idaviso', how='left')
vistas_por_nivel=vistas_por_nivel.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})
vistas_por_nivel.head()

,idpostulante,nivel_laboral,vistas_por_nivel
0,0005E,Otro,2
1,00Lkv,Otro,2
2,00dMd,Otro,4
3,01QAq,Otro,1
4,021OM,Otro,5


In [12]:
vistas_por_tipo=vistas.copy()
vistas_por_tipo['vistas_por_tipo']=0;
vistas_por_tipo=pd.merge(vistas_por_tipo,avisos,on='idaviso', how='left')
vistas_por_tipo=vistas_por_tipo.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_por_tipo.head()

,idpostulante,tipo_de_trabajo,vistas_por_tipo
0,0005E,Full-time,2
1,00Lkv,Full-time,2
2,00dMd,Full-time,4
3,01QAq,Full-time,1
4,021OM,Full-time,5


In [38]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [39]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,15
1,0005E,58
2,000R8,14
3,001XE,26
4,003k9,32


In [40]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')

In [41]:
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,2,28
1,48375,RwVdKR,1,15
2,169730,1KjXB,2,99
3,169730,2AKzxa,2,2
4,169730,6LJ64,1,23


In [42]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.071429
1,48375,RwVdKR,0.066667
2,169730,1KjXB,0.020202
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.043478


In [43]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})

In [19]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/12.csv")
postulaciones2=postulaciones2.append(pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/8.csv").sample(len(postulaciones2)//2).reset_index(drop=True))
print(len(postulaciones2))
postulaciones2

9906801


,idaviso,idpostulante,sepostulo
0,1112334662,e3K864,0
1,1112339426,6rQORvO,0
2,1112406702,ZD2waX1,0
3,1112452620,YjAvKQV,0
4,1112428719,Ee6wR2,0
5,1112420544,6rMdGN4,0
6,1112485812,96XkRRZ,0
7,1112501858,ak5YvVE,0
8,1112202829,5xmK8p,0
9,1112205371,LNKJBLp,0


In [20]:
postulaciones.head()

,idaviso,idpostulante,sepostulo
0,1112257047,NM5M,1
1,1111920714,NM5M,1
2,1112346945,NM5M,1
3,1112345547,NM5M,1
4,1112237522,5awk,1


In [21]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones2=0
gc.collect()
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

16511335
16502304


In [22]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad
15429626,1111452551,JBXED15,0,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,True,False,False,False,NaN,NaN,NaN
6946465,1112438851,OqbMRpx,0,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,48.0,NaN,NaN
3702600,1112261253,5mlA65M,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,15.0,NaN,NaN
7946344,1112379341,LNKomA5,1,MASC,21.0,-,-,Graduado,-,En Curso,...,False,False,False,False,False,False,False,4.0,NaN,NaN
1691753,1112381835,BmDOqEQ,0,FEM,29.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,2.0,NaN,NaN
6085708,1112216935,6EEjGL,0,MASC,47.0,-,-,-,-,-,...,True,False,False,False,False,False,False,40.0,NaN,NaN
1352462,1112291114,X95Q60O,1,FEM,28.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,1.0,NaN,NaN
3255085,1759000,MV654rN,0,FEM,200.0,-,-,-,-,-,...,False,False,False,True,False,False,False,NaN,NaN,NaN
4533577,1111857718,bO4b2GB,0,MASC,41.0,-,-,-,-,Graduado,...,True,False,False,False,False,False,False,12.0,NaN,NaN
8084238,1112276785,pze1KXW,0,FEM,29.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,51.0,NaN,NaN


In [23]:
postulaciones['cantidad']=postulaciones['cantidad'].fillna(0)
postulaciones['vistas_postulante']=postulaciones['vistas_postulante'].fillna(0)
postulaciones['vistas_al_aviso']=postulaciones['vistas_al_aviso'].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,vistas_postulante,vistas_al_aviso,cantidad,lo_vio
2752133,1112371208,xkdxAKz,0,MASC,20.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
1582845,1112033906,vV98vAo,1,MASC,19.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,0.0,0.0,0.000000,False
12670812,1111969652,pzdQ09m,0,FEM,21.0,-,-,-,-,-,...,False,False,False,False,False,False,2.0,0.0,0.000000,False
4109947,1112313656,JBeQZPb,1,MASC,24.0,-,-,-,Graduado,Graduado,...,False,False,True,False,True,False,164.0,0.0,0.000000,False
2227129,1112414507,pzeJNxZ,1,MASC,27.0,-,-,-,-,Graduado,...,False,False,False,False,True,False,38.0,1.0,0.026316,True
5013645,1112210610,Y2boE1,0,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,0.0,0.0,0.000000,False
13783135,1111371458,Pma8kE1,0,FEM,22.0,-,-,-,-,Graduado,...,False,False,True,False,True,False,0.0,0.0,0.000000,False
15975138,1112009186,6zAX5x,0,FEM,39.0,-,-,-,-,-,...,False,False,False,False,False,False,10.0,0.0,0.000000,False
10172754,1112418341,8MaQjP3,1,FEM,24.0,-,-,-,-,-,...,False,False,False,False,False,False,4443.0,1.0,0.000225,True
9599603,1112438516,JBrRerk,0,FEM,34.0,-,-,-,-,-,...,False,False,False,False,False,False,4.0,0.0,0.000000,False


In [25]:
postulaciones_por_area=postulaciones.copy()
postulaciones_por_area=postulaciones_por_area.groupby(by=['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_area=postulaciones_por_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_por_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Administración,1
1,0z5Dmrd,Almacén / Depósito / Expedición,1
2,0z5Dmrd,Compensación y Planilla,1
3,0z5Dmrd,Consultoria,1
4,0z5Dmrd,Contabilidad,1


In [26]:
postulaciones_por_nivel=postulaciones.copy()
postulaciones_por_nivel=postulaciones_por_nivel.groupby(by=['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_nivel=postulaciones_por_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})
postulaciones_por_nivel.head()

,idpostulante,nivel_laboral,postulaciones_mismo_nivel
0,0z5Dmrd,-,1
1,0z5Dmrd,Gerencia / Alta Gerencia / Dirección,1
2,0z5Dmrd,Otro,1
3,0z5Dmrd,Senior / Semi-Senior,17
4,0z5JW1r,-,1


In [27]:
postulaciones_por_tipo=postulaciones.copy()
postulaciones_por_tipo=postulaciones_por_tipo.groupby(by=['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_por_tipo=postulaciones_por_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_por_tipo.head()

,idpostulante,tipo_de_trabajo,postulaciones_mismo_tipo
0,0z5Dmrd,Full-time,20
1,0z5JW1r,Full-time,27
2,0z5JW1r,Part-time,3
3,0z5JW1r,Por Horas,1
4,0z5JW1r,Teletrabajo,1


In [30]:
postulaciones=pd.merge(postulaciones,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones=pd.merge(postulaciones,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
2541554,1112405355,81PNJx,1,FEM,30.0,-,-,-,-,Graduado,...,21.0,2.0,0.095238,True,0,6,5,2.0,4.0,4.0
12120442,1112411607,5mlW3lN,1,MASC,27.0,-,-,En Curso,-,Graduado,...,12.0,0.0,0.000000,False,0,43,22,0.0,12.0,12.0
9132879,1112041659,4rdOv5P,0,MASC,26.0,-,-,-,-,Graduado,...,41.0,0.0,0.000000,False,1,22,13,1.0,22.0,22.0
14989682,1112061712,amwZOZ,0,FEM,33.0,-,-,-,-,-,...,9.0,0.0,0.000000,False,1,21,14,0.0,9.0,9.0
1266648,12543760,EQleQJ,1,MASC,29.0,-,-,Graduado,-,Graduado,...,0.0,0.0,0.000000,False,6,31,25,0.0,0.0,0.0
11489656,1112474112,E908B0,0,MASC,51.0,-,-,-,-,-,...,10.0,0.0,0.000000,False,0,22,1,0.0,8.0,8.0
14671123,1112383956,W9V4Pwa,0,FEM,19.0,-,-,-,-,Graduado,...,16.0,0.0,0.000000,False,1,16,14,1.0,13.0,13.0
9462361,1112250613,X9PeQxv,0,FEM,22.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,1,18,14,0.0,0.0,0.0
10631863,1112448012,969VNVZ,0,MASC,34.0,-,-,-,-,Graduado,...,0.0,0.0,0.000000,False,0,43,39,0.0,0.0,0.0
7515792,1001135716,ZDa6QWE,0,FEM,27.0,-,-,Graduado,-,-,...,16.0,0.0,0.000000,False,2,15,12,0.0,16.0,16.0


In [31]:
train, test=train_test_split(postulaciones,test_size=0.05)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0

Train:  15282156 Test:  804325


In [32]:
gc.collect()

1025

In [33]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=6000,max_depth=13,n_estimators=1200,colsample_bytree=0.8,n_jobs=-1,random_state=0,silent=False,subsample=0.8,sumsample_freq=0.5)

dt=c.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=1)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl")

[1]	valid_0's binary_logloss: 0.686237
Training until validation scores don't improve for 1 rounds.
[2]	valid_0's binary_logloss: 0.679655
[3]	valid_0's binary_logloss: 0.672855
[4]	valid_0's binary_logloss: 0.666017
[5]	valid_0's binary_logloss: 0.659388
[6]	valid_0's binary_logloss: 0.65314
[7]	valid_0's binary_logloss: 0.646835
[8]	valid_0's binary_logloss: 0.64049
[9]	valid_0's binary_logloss: 0.634413
[10]	valid_0's binary_logloss: 0.628599
[11]	valid_0's binary_logloss: 0.62259
[12]	valid_0's binary_logloss: 0.616681
[13]	valid_0's binary_logloss: 0.61094
[14]	valid_0's binary_logloss: 0.605848
[15]	valid_0's binary_logloss: 0.600614
[16]	valid_0's binary_logloss: 0.595701
[17]	valid_0's binary_logloss: 0.59027
[18]	valid_0's binary_logloss: 0.58536
[19]	valid_0's binary_logloss: 0.580099
[20]	valid_0's binary_logloss: 0.575299
[21]	valid_0's binary_logloss: 0.570224
[22]	valid_0's binary_logloss: 0.565285
[23]	valid_0's binary_logloss: 0.560728
[24]	valid_0's binary_logloss: 0.5

In [34]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=c.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9784332454342526


In [35]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBMV25.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBMV25.pkl']

In [44]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion['lo_vio']=(prediccion['cantidad']>0)
prediccion=pd.merge(prediccion,postulaciones_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
prediccion=pd.merge(prediccion,vistas_por_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_por_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_por_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
32441,30177,1112377373,rm2kNa1,FEM,20.0,-,-,-,-,Graduado,...,42.0,1.0,0.023810,True,18.0,13.0,8.0,14.0,10.0,10.0
85627,41038,1112420433,wVkvPrQ,MASC,24.0,-,-,-,-,-,...,16.0,1.0,0.062500,True,1.0,11.0,8.0,4.0,9.0,9.0
44475,52754,1112443958,a54dD9,MASC,32.0,-,-,-,-,-,...,350.0,1.0,0.002857,True,93.0,190.0,172.0,82.0,159.0,159.0
85684,34898,1112409522,eGY6D2,FEM,29.0,-,-,Graduado,-,Graduado,...,NaN,NaN,0.000000,False,0.0,2.0,1.0,0.0,0.0,0.0
32052,90324,1112467373,1eqzzz,MASC,31.0,-,-,-,-,Graduado,...,30.0,NaN,0.000000,False,0.0,11.0,0.0,0.0,18.0,18.0
91695,90570,1112468132,Yaq2eq,FEM,26.0,-,-,-,-,-,...,73.0,NaN,0.000000,False,0.0,33.0,20.0,0.0,63.0,63.0
64666,11588,1111959911,1Z929b,MASC,38.0,-,-,-,-,-,...,39.0,NaN,0.000000,False,0.0,0.0,0.0,0.0,7.0,7.0
89617,51161,1112441060,aPpaWZ,FEM,48.0,-,-,-,-,-,...,1.0,NaN,0.000000,False,0.0,7.0,3.0,0.0,1.0,1.0
72759,78072,1112464476,aXL0a5,FEM,30.0,-,-,Graduado,-,-,...,14.0,1.0,0.071429,True,0.0,7.0,1.0,1.0,8.0,8.0
42636,67163,1112460175,RzPLzRx,FEM,21.0,-,-,-,-,Graduado,...,16.0,1.0,0.062500,True,5.0,1.0,2.0,6.0,3.0,3.0


In [45]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.86030115, 0.13969885],
       [0.97129031, 0.02870969],
       [0.8825703 , 0.1174297 ],
       ...,
       [0.98941296, 0.01058704],
       [0.98864589, 0.01135411],
       [0.99473232, 0.00526768]])

In [46]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [47]:
prediccion['sepostulo']=y_final2

In [48]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,NaN,0.0,False,0.0,9.0,0.0,4.0,24.0,24.0,0.139699
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,NaN,0.0,False,4.0,51.0,0.0,0.0,6.0,6.0,0.028710
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,NaN,0.0,False,0.0,9.0,0.0,1.0,7.0,7.0,0.117430
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,NaN,0.0,False,0.0,6.0,1.0,0.0,0.0,0.0,0.002372
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,NaN,0.0,False,0.0,2.0,0.0,0.0,3.0,3.0,0.001747


In [49]:
prediccion=prediccion[['id','sepostulo']]
prediccion.head()

,id,sepostulo
0,0,0.139699
1,1,0.028710
2,2,0.117430
3,3,0.002372
4,4,0.001747


In [50]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion56.csv",index=False)

In [51]:
prediccion.sepostulo.mean()

0.45802513492309177